In [1]:
import torch
from torch import optim, nn
from torchvision import models, transforms
import numpy as np
from torchvision import datasets, transforms
from tqdm import tqdm
import matplotlib.pyplot as plt
model = models.vgg16(pretrained=True)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
class FeatureExtractor(nn.Module):
  def __init__(self, model):
    super(FeatureExtractor, self).__init__()
		# Extract VGG-16 Feature Layers
    self.features = list(model.features)
    self.features = nn.Sequential(*self.features)
		# Extract VGG-16 Average Pooling Layer
    self.pooling = model.avgpool
		# Convert the image into one-dimensional vector
    self.flatten = nn.Flatten()
		# Extract the first part of fully-connected layer from VGG16
    self.fc = model.classifier[0]
  
  def forward(self, x):
		# It will take the input 'x' until it returns the feature vector called 'out'
    out = self.features(x)
    out = self.pooling(out)
    out = self.flatten(out)
    out = self.fc(out) 
    return out 

# Initialize the model
model = models.vgg16(pretrained=True)
new_model = FeatureExtractor(model)

# Change the device to GPU
device = torch.device('cuda:1' if torch.cuda.is_available() else "cpu")
new_model = new_model.to(device)

In [6]:
transform = transforms.Compose([transforms.Resize((32,32)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))
                               ])

trainset = datasets.FashionMNIST('MNIST_data/', download = True, train = True, transform = transform)
testset = datasets.FashionMNIST('MNIST_data/', download = True, train = False, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 1, shuffle = True)
testloader = torch.utils.data.DataLoader(testset, batch_size = 1, shuffle = True)

In [7]:
next(iter(trainloader))[0].size()

torch.Size([1, 1, 32, 32])

In [19]:
features = []
for image, label in tqdm(trainset):
    # img = transform(image[0][0])
    img = torch.tensor(np.array([np.stack((image[0].cpu(),)*3, axis=-1)]))
    img = img.permute(0,3,1,2)
    # print(img.size())
    img = img.to(device)
    with torch.no_grad():
		# Extract the feature from the image
        feature = new_model(img)
	# Convert to NumPy Array, Reshape it, and save it to features variable
    features.append(feature.cpu().detach().numpy().reshape(-1))
features = np.array(features)

100%|██████████| 60000/60000 [13:29<00:00, 74.16it/s] 


In [20]:
features.shape

(60000, 4096)

In [21]:
import pickle

In [22]:
save_path = "features.npy"
with open(save_path, 'wb') as f:
    pickle.dump(features, f)